# SpecAugment with TorchAudio
by [Zach Caceres](https://github.com/zcaceres) and [Jenny Cai](https://github.com/qcai2002)

This notebook feature a Pytorch implementation of the data augmentation techniques in [SpecAugment](https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html) by Google Brain.

The techniques are:
- Time Warp
- Frequency Mask
- Time Mask

SpecAugment reports that Frequency Mask and Time Mask were the most effective augmentations and less computationally expensive than Time Warp. We included Time Warp for completeness.

### Prerequisites
Be sure you've run `install.sh` before running this notebook!

### Setup

In [1]:
#Export
from collections import namedtuple
import random

import torch
import torchaudio
from torchaudio import transforms

ModuleNotFoundError: No module named 'torchaudio'

This converts our implementation of `sparse_image_warp` into a Python file we can import.

In [ ]:
%%sh
python notebook2script.py SparseImageWarp.ipynb

In [ ]:
from exp.nb_SparseImageWarp import sparse_image_warp

## Loading Audio

Let's check out our sample wav file (a snippet of background chatter from a party). We'll load it into a `namedtuple` to keep the signal & sample rate tied together.

In [ ]:
from IPython.display import Audio

In [ ]:
sample = './party-crowd.wav'

In [ ]:
AudioData = namedtuple('AudioData', ['sig', 'sr'])

In [ ]:
audio = AudioData(*torchaudio.load(sample))

In [ ]:
audio

In [ ]:
def check_audio(aud):
    display(Audio(data=aud.sig, rate=aud.sr))

In [ ]:
check_audio(audio)

## Wav to (Mel)Spectrogram

Simple wrapper for `torchaudio`'s transforms to make a decibel-scale Melspectrogram. Borrowed from [fastai-audio](https://github.com/zcaceres/fastai-audio), with some tweaks.

In [ ]:
def tfm_spectro(ad:Audio, sr=16000, to_db_scale=False, n_fft=1024, 
                ws=None, hop=None, f_min=0.0, f_max=-80, pad=0, n_mels=128):
    # We must reshape signal for torchaudio to generate the spectrogram.
    mel = transforms.MelSpectrogram(sr=ad.sr, n_mels=n_mels, n_fft=n_fft, ws=ws, hop=hop, 
                                    f_min=f_min, f_max=f_max, pad=pad,)(ad.sig.reshape(1, -1))
    mel = mel.permute(0,2,1) # swap dimension, mostly to look sane to a human.
    if to_db_scale: mel = transforms.SpectrogramToDB(stype='magnitude', top_db=f_max)(mel)
    return mel

In [ ]:
spectro = tfm_spectro(audio, ws=512, hop=256, n_mels=128, to_db_scale=True, f_max=8000, f_min=-80)

In [ ]:
# # Run this cell if you want to use your spectrogram to run the SparseImageWarp notebook.
# %store spectro

## Viewing a spectrogram

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def tensor_to_img(spectrogram):
    plt.figure(figsize=(14,1)) # arbitrary, looks good on my screen.
    plt.imshow(spectrogram[0])
    plt.show()
    display(spectrogram.shape)

In [ ]:
tensor_to_img(spectro)

## Time Warp

Time Warp is described as:
> Time warping is applied via the function sparse image warp of tensorflow. Given a log mel spectrogram with τ time steps, we view it as an image where the time axis is horizontal and the frequency axis is vertical. A random point along the horizontal line passing through the center of the image within the time steps (W, τ − W) is to be warped either to the left or right by a distance w chosen from a uniform distribution from 0 to the time warp parameter W along that line.

Intuitively we squish/stretch the audio in a given direction bounded by the `W` parameter we select.

NOTE:
`sparse_image_warp` does not exist in Pytorch, so you can see our implementation in the SparseImageWarp.ipynb notebook. Our implementation skips certain features like add clamping.

You should be careful with your `W` parameter because it's highly dependent on your audio and spectrogram. 

*Sanity check your spectrograms if you're using this transform.*

In [ ]:
#Export
def time_warp(spec, W=5):
    num_rows = spec.shape[1]
    spec_len = spec.shape[2]
    device = spec.device

    # adapted from https://github.com/DemisEom/SpecAugment/
    pt = (num_rows - 2* W) * torch.rand([], dtype=torch.int) + W # random point along the time axis
    src_ctr_pt_freq = torch.range(0, spec_len // 2)  # control points on freq-axis
    src_ctr_pt_time = torch.ones_like(src_ctr_pt_freq) * pt  # control points on time-axis
    src_ctr_pts = torch.stack((src_ctr_pt_time, src_ctr_pt_freq), dim=-1)
    src_ctr_pts = src_ctr_pts.float().to(device)

    # Destination
    w = 2 * W * torch.rand([], dtype=torch.int) - W# distance
    dest_ctr_pt_freq = src_ctr_pt_freq
    dest_ctr_pt_time = src_ctr_pt_time + w
    dest_ctr_pts = torch.stack((dest_ctr_pt_time, dest_ctr_pt_freq), dim=-1)
    dest_ctr_pts = dest_ctr_pts.float().to(device)

    # warp
    source_control_point_locations = torch.unsqueeze(src_ctr_pts, 0)  # (1, v//2, 2)
    dest_control_point_locations = torch.unsqueeze(dest_ctr_pts, 0)  # (1, v//2, 2)

    warped_spectro, dense_flows = sparse_image_warp(spec, source_control_point_locations, dest_control_point_locations)
    return warped_spectro.squeeze(3)

In [ ]:
def test_time_warp():
    tensor_to_img(time_warp(spectro))
tensor_to_img(spectro)    
test_time_warp()

## Frequency Mask

Frequency masking is described as:
> Frequency masking is applied so that f consecutive mel
frequency channels [f0, f0 + f) are masked, where f is first chosen from a uniform distribution from 0 to the frequency mask parameter F, and f0 is chosen from 0, ν − f). ν is the number of mel frequency channels.

Intuitively, this is similar to cutout in computer vision workflows. We mask certain frequency bands with either the mean value of the spectrogram or zero.

In [ ]:
#Export
def freq_mask(spec, F=30, num_masks=1, replace_with_zero=False):
    cloned = spec.clone()
    num_mel_channels = cloned.shape[1]
    
    for i in range(0, num_masks):        
        f = random.randrange(0, F)
        f_zero = random.randrange(0, num_mel_channels - f)

        # avoids randrange error if values are equal and range is empty
        if (f_zero == f_zero + f): return cloned

        mask_end = random.randrange(f_zero, f_zero + f) 
        if (replace_with_zero): cloned[0][f_zero:mask_end] = 0
        else: cloned[0][f_zero:mask_end] = cloned.mean()
    
    return cloned

In [ ]:
def test_freq_mask():
    tensor_to_img(freq_mask(spectro))
    # Two Masks...
    tensor_to_img(freq_mask(spectro, num_masks=2))
    # with zeros
    tensor_to_img(freq_mask(spectro, num_masks=2, replace_with_zero=True))
test_freq_mask()

## Time Mask

Time masking is described as:
> Time masking is applied so that t consecutive time steps
[t0, t0 + t) are masked, where t is first chosen from a uniform distribution from 0 to the time mask parameter T, and t0 is chosen from [0, τ − t). We introduce an upper bound on the time mask so that a time mask cannot be wider than p times the number of time steps.

Intuitively, this is similar to cutout and the frequency mask above. We mask certain time ranges with the mean value of the spectrogram or zero.

In [ ]:
#Export
def time_mask(spec, T=40, num_masks=1, replace_with_zero=False):
    cloned = spec.clone()
    len_spectro = cloned.shape[2]
    
    for i in range(0, num_masks):
        t = random.randrange(0, T)
        t_zero = random.randrange(0, len_spectro - t)

        # avoids randrange error if values are equal and range is empty
        if (t_zero == t_zero + t): return cloned

        mask_end = random.randrange(t_zero, t_zero + t)
        if (replace_with_zero): cloned[0][:,t_zero:mask_end] = 0
        else: cloned[0][:,t_zero:mask_end] = cloned.mean()
    return cloned

In [ ]:
def test_time_mask():
    tensor_to_img(time_mask(spectro))
    # Two Masks...
    tensor_to_img(time_mask(spectro, num_masks=2))
    # with zeros
    tensor_to_img(time_mask(spectro, num_masks=2, replace_with_zero=True))
test_time_mask()

## Combined
Here we combine all three transforms.

In [ ]:
combined = time_mask(freq_mask(time_warp(spectro), num_masks=2), num_masks=2)
tensor_to_img(combined)

## Performance

### CPU

In [ ]:
%%timeit -n 10
time_warp(spectro)

In [ ]:
%%timeit -n 100
freq_mask(spectro)

In [ ]:
%%timeit -n 100
time_mask(spectro)

In [ ]:
%%timeit -n 10
time_mask(freq_mask(time_warp(spectro), num_masks=2), num_masks=2)

### GPU

In [ ]:
cspectro = spectro.cuda()

In [ ]:
%%timeit -n 100
time_warp(cspectro)

In [ ]:
%%timeit -n 100
freq_mask(cspectro)

In [ ]:
%%timeit -n 100
time_mask(cspectro)

In [ ]:
%%timeit -n 100
time_mask(freq_mask(time_warp(cspectro), num_masks=2), num_masks=2)